# Example 1: Scalar equation

## The ODE

Let's solve:

$y'(t) = \cos(t^2)\sqrt{y}$ for $t \in [0, 10]$,

with initial condition $y_0 = y(0) = 1$.

In this case, we have $f(y,t) = \cos(t^2)\sqrt{y}$

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# f(y,t)
f = lambda y, t: np.cos(t**2)*np.sqrt(y)

# Test it out: f(4, 0) = 2
print(f(4, 0))

## Time step setup

We use the following definitions
* $N$ is the number of time steps for the algorithm to take
* $T$ is the end time of the simulation, in this case, $T = 10$
* $h$ is the time step size
* The relationship $T = Nh$ always holds as long as time step remains constant throughout the simulation

We must choose two of the three values for $T$, $N$, and $h$; the third is set by the relation $T = Nh$. 

Below are two options for setting up the time discretization. Option 1 chooses number of time steps and sets $h$ accordingly. Option two choose the time step size and sets $N$ accordingly. Both options assume an end time of $T = 10$.

In [ ]:
# Time step setup: define number of steps to take (option 1)
T = 10      # End time
N = 1000    # Number of steps to take
h = T/N     # Time step size
t = np.linspace(0, T, N+1)
print('h = T/N = ', h)
print('t = ', t)

# Time step setup: define time step (option 2)
T = 10      # End time
h = 0.01    # Time step size
t = np.arange(0, T+h, h)
N = len(t)-1
print('N = T*h = ', N)
print('t = ', t)

## Set up the solution vector y

<code>y</code> will store the numerical solution at each time point, with the initial condition as its first entry. 

While Python/numpy allows you to start with an empty array and append entries to it, it is good practice to pre-allocate <code>y</code> to be filled with zeros. It will be the same size as <code>t</code>, with $N+1$ entries

In [ ]:
# Define the initial condition
y0 = 1

# Initialize y as an array os N+1 zeros
y = np.zeros(N+1)

# Set the first entry in y to be the initial condition
y[0] = y0

## Time step loop

Now let's loop on the time steps and do the forward Euler algorithm



In [ ]:
# Time step loop
# Using 'range' like this will set n = 0, 1, 2, ..., N-1
# This ensures that in the last step, y[N] is computed, which is the desired result
for n in range(N):
    y[n+1] = y[n] + h*f(y[n], t[n])
    

## Plot the result

Now we'll plot the result. We want to plot <code>y</code>, the which represents the solution at each time $t_n$ for $n = 0, 1, 2, ... N$, versus the variable storing the $t_n$ values, <code>t</code>

In [ ]:
# Plotting
plt.plot(t, y)
plt.grid()
plt.xlabel('t')
plt.ylabel('y')
plt.title(r'$y(t)$ numerical solution')
plt.show()

## Solution convergence

We generally want to pick a time step that is small enough to give an accurate solution result but big enough to not incur unecessary cost.

Often, we cannot know the optimal step size $h$ in advance. A typical approach is to try several values of $h$ to verify that the solution has converged, or stopped notably changing, with further decreases in $h$.

This is most readily accomplished by using a function to complete perform the time integration for a given step size, then calling that function several times to produce the solution. This allows easy plotting of several solutions for direct comparison.

Below, the forward Euler code is refactored into a function, and results using different step sizes are plotted. What appears to be a good choice of step size $h$?

In [ ]:
def fwd_euler_scalar_example(h):

    # ODE right hand side
    f = lambda y, t: np.cos(t**2)*np.sqrt(y)

    # Time discretization setup
    T = 10
    t = np.arange(0, T+h, h)
    N = len(t)-1

    # Initialize condition
    y0 = 1
    y = np.zeros(N+1)
    y[0] = y0

    for n in range(N):
        y[n+1] = y[n] + h*f(y[n], t[n])

    return y, t


# Plot solution for several values of h
hvals = [0.1, 0.05, 0.02, 0.01]
plt.figure(figsize=(8,6))
for h in hvals:
    y, t = fwd_euler_scalar_example(h)
    plt.plot(t, y, label=f'h = {h}')
plt.grid()
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.title('Solutions for several h values')
plt.show()

# Example 2: system of equations

Let's solve the following system of $m = 3$ equations

$$\mathbf{y} = (y_1, y_2, y_3)$$

$$y'_1 = y_1 + y_2 + t$$

$$y'_2 = \dfrac{-y_3^2}{1+t}$$

$$y'_3 = -y_1$$

with initial condition $\mathbf{y_0} = (1, -1, 0.5)$

Note that we could also write this more succinctly as

$$\mathbf{y}' = \mathbf{f}(\mathbf{y}, t) = \left[\array{y_1 + y_2 + t \\ \dfrac{-y_3^2}{1+t} \\ -y_1}\right]$$

In [ ]:
# This lambda function represents a vector equation with the vector RHS f(y,t)
f = lambda y, t: np.array([y[0] + y[1] + t, -y[2]**2/(1+t), -y[0]])

# Test out f: f([0, 1, 2], 1) = [2, -2, 0]
ytest = np.array([0, 1, 2])
ttest = 1
print(f(ytest, ttest))

## Forward Euler

Below we implement forward Euler. This time, the solution variable <code>y</code> will be a <b>2D numpy array</b> of dimension $3\times (N+1)$.
* Each row in <code>y</code> represents one of the solutions $y_i(t)$. For example, <code>y[0,:]</code> represents $y_1(t)$, and is a 1D numpy array with $N+1$ entries
* Each column in <code>y</code> represents all three solutions at a particular time. For example, <code>y[:,2]</code> represents $(y_1, y_2, y_3)$ at time $t_2 = 2h$, and is a 1D numpy array with $3$ entries

This means the solution vector <code>y</code> has the following form:

$$
\begin{bmatrix}
y_1(t_0) & y_1(t_1) & y_1(t_2) & \cdots & y_1(t_N) \\
y_2(t_0) & y_2(t_1) & y_2(t_2) & \cdots & y_2(t_N) \\
y_3(t_0) & y_3(t_1) & y_3(t_2) & \cdots & y_3(t_N)
\end{bmatrix}
$$

In the time step loop, we need to update each of $y_1$, $y_2$, and $y_3$. This can be accomplished with one line inside the loop by taking advantage of numpy's ability to vectorize array addition.

In [ ]:
# Time step setup
T = 4
N = 40
h = T/N
t = np.linspace(0, T, N+1)

# Define the initial condition
y0 = np.array([1, -1, 0.5])

# y is 3 rows, N+1 columns
# Each column stores the solution (y1, y2, y3) at a particular time
y = np.zeros((3,N+1))

# Set the initial condition in the solution variable y
y[:,0] = y0

# Forward Euler loop
for n in range(N):
    # Update using vectorized array addition
    y[:,n+1] = y[:,n] + h*f(y[:,n], t[n])


## Plotting

To plot each solution versus time, we want to plot each of the rows of <code>y</code> against the variable <code>t</code>. Since <code>y</code> contains three rows, we need to call the plotting command <code>plt.plot</code> three times. To make our plot readable, we add a label for each plotting command and add a legend as well.

In [ ]:
# Plotting
plt.plot(t, y[0,:], '.-', label=r'$y_1(t)$')
plt.plot(t, y[1,:], '*-',  label=r'$y_2(t)$')
plt.plot(t, y[2,:], 'o-', label=r'$y_3(t)$')
plt.grid()
plt.xlabel('t')
plt.ylabel('y')
plt.title(r'$y(t)$ numerical solution')
plt.legend()
plt.show()